# 16_new_6th_test 요약

- 모델: SimpleMLPWithEmbedding (딥러닝 Only)
- 변경점: pos_weight train 기준, drop_last=False, TE smoothing 강화, QuantileTransformer 사용
- 학습/평가: StratifiedKFold 5, seed=777, ROC-AUC 기준
- 제출파일: submission_16_new_6th_test.csv


## 1. 라이브러리 로드


In [6]:
import os
import random
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder, QuantileTransformer
from sklearn.metrics import roc_auc_score

# ============================================================
# CONFIG
# ============================================================



## 2. 설정값 및 시드 고정


In [7]:
SEED = 777
N_FOLDS = 5
BATCH_SIZE = 512
EPOCHS = 50
PATIENCE = 8
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"🖥️ Device: {DEVICE}")
print(f"📊 Config: {N_FOLDS}-Fold, {EPOCHS} epochs, batch={BATCH_SIZE}")


def set_seed(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


set_seed(SEED)

# ============================================================
# 1. 데이터 로딩
# ============================================================
print("\n" + "=" * 60)
print("📂 1. 데이터 로딩")
print("=" * 60)



🖥️ Device: cpu
📊 Config: 5-Fold, 50 epochs, batch=512

📂 1. 데이터 로딩


## 3. 데이터 로드


In [8]:
train_raw = pd.read_csv("../../data/raw/train.csv")
test_raw = pd.read_csv("../../data/raw/test_x.csv")

print(f"Train: {train_raw.shape}, Test: {test_raw.shape}")

train_raw["voted_bin"] = (train_raw["voted"] == 2).astype(int)
print(f"타겟 분포: 투표안함={train_raw['voted_bin'].mean():.1%}")

# ============================================================
# 2. 이상치/무응답 처리
# ============================================================
print("\n" + "=" * 60)
print("🔧 2. 이상치/무응답 처리 함수 정의")
print("=" * 60)


def clean_data(df, is_train=True):
    df = df.copy()

    # 2-1. 무응답 처리 (0 → NaN)
    zero_to_nan_cols = ["education", "engnat", "hand", "married", "urban"]
    for col in zero_to_nan_cols:
        if col in df.columns:
            df.loc[df[col] == 0, col] = np.nan

    # 2-2. familysize 처리
    if "familysize" in df.columns:
        df.loc[df["familysize"] == 0, "familysize"] = np.nan
        df.loc[df["familysize"] > 15, "familysize"] = np.nan

    # 2-3. TP 성격 처리 (0 → NaN)
    tp_cols = [f"tp{i:02d}" for i in range(1, 11)]
    for col in tp_cols:
        if col in df.columns:
            df.loc[df[col] == 0, col] = np.nan

    # 2-4. Q_E 응답시간 클리핑 (100ms ~ 60초)
    qe_cols = [f"Q{c}E" for c in "abcdefghijklmnopqrst"]
    for col in qe_cols:
        if col in df.columns:
            df[col] = df[col].clip(lower=100, upper=60000)

    return df


# ============================================================
# 3. 피처 엔지니어링
# ============================================================
print("\n" + "=" * 60)
print("🔨 3. 피처 엔지니어링 함수 정의")
print("=" * 60)



Train: (45532, 78), Test: (11383, 77)
타겟 분포: 투표안함=54.7%

🔧 2. 이상치/무응답 처리 함수 정의

🔨 3. 피처 엔지니어링 함수 정의


## 4. 피처 생성 함수


In [9]:
def build_features(df, is_train=True):
    df = df.copy()

    # 3-1. 인구통계 피처
    age_map = {"10s": 1, "20s": 2, "30s": 3, "40s": 4, "50s": 5, "60s": 6, "+70s": 7}
    df["age_ord"] = df["age_group"].map(age_map)

    df["is_teenager"] = (df["age_ord"] == 1).astype(int)
    df["is_young"] = (df["age_ord"] <= 2).astype(int)
    df["is_middle"] = ((df["age_ord"] >= 3) & (df["age_ord"] <= 5)).astype(int)
    df["is_old"] = (df["age_ord"] >= 6).astype(int)

    df["edu_low"] = (df["education"] <= 2).astype(float)
    df["edu_high"] = (df["education"] >= 3).astype(float)

    df["is_single"] = (df["married"] == 1).astype(float)
    df["is_married"] = (df["married"] == 2).astype(float)

    df["is_urban"] = (df["urban"] == 3).astype(float)
    df["is_rural"] = (df["urban"] == 1).astype(float)

    df["is_english_native"] = (df["engnat"] == 1).astype(float)
    df["is_male"] = (df["gender"] == "Male").astype(int)

    # 3-2. Q_A
    qa_cols = [f"Q{c}A" for c in "abcdefghijklmnopqrst"]

    df["qa_mean"] = df[qa_cols].mean(axis=1)
    df["qa_std"] = df[qa_cols].std(axis=1)
    df["qa_min"] = df[qa_cols].min(axis=1)
    df["qa_max"] = df[qa_cols].max(axis=1)
    df["qa_range"] = df["qa_max"] - df["qa_min"]

    df["qa_extreme_ratio"] = ((df[qa_cols] == 1) | (df[qa_cols] == 5)).sum(axis=1) / 20
    df["qa_neutral_ratio"] = (df[qa_cols] == 3).sum(axis=1) / 20
    df["qa_negative_ratio"] = (df[qa_cols] <= 2).sum(axis=1) / 20
    df["qa_positive_ratio"] = (df[qa_cols] >= 4).sum(axis=1) / 20

    df["qa_all_same"] = (df[qa_cols].std(axis=1) == 0).astype(int)

    # 3-3. Q_E (log)
    qe_cols = [f"Q{c}E" for c in "abcdefghijklmnopqrst"]
    for col in qe_cols:
        df[f"{col}_log"] = np.log1p(df[col])

    qe_log_cols = [f"{col}_log" for col in qe_cols]

    df["qe_log_mean"] = df[qe_log_cols].mean(axis=1)
    df["qe_log_std"] = df[qe_log_cols].std(axis=1)
    df["qe_log_min"] = df[qe_log_cols].min(axis=1)
    df["qe_log_max"] = df[qe_log_cols].max(axis=1)

    df["qe_fast_ratio"] = (df[qe_cols] < 500).sum(axis=1) / 20
    df["qe_slow_ratio"] = (df[qe_cols] > 10000).sum(axis=1) / 20
    df["qe_total_log"] = df[qe_log_cols].sum(axis=1)

    df["is_careless"] = ((df[qe_cols].mean(axis=1) < 500) | (df["qa_all_same"] == 1)).astype(int)

    # 3-4. TP Big5
    tp_cols = [f"tp{i:02d}" for i in range(1, 11)]
    df["tp_missing_ratio"] = df[tp_cols].isna().sum(axis=1) / 10

    df["extraversion"] = df["tp01"] - df["tp06"]
    df["agreeableness"] = df["tp07"] - df["tp02"]
    df["conscientiousness"] = df["tp03"] - df["tp08"]
    df["neuroticism"] = df["tp04"] - df["tp09"]
    df["openness"] = df["tp05"] - df["tp10"]

    df["tp_mean"] = df[tp_cols].mean(axis=1)
    df["tp_std"] = df[tp_cols].std(axis=1)

    # 3-5. WR/WF
    wr_cols = [f"wr_{i:02d}" for i in range(1, 14)]
    wf_cols = [f"wf_{i:02d}" for i in range(1, 4)]

    df["wr_sum"] = df[wr_cols].sum(axis=1)
    df["wf_sum"] = df[wf_cols].sum(axis=1)
    df["word_credibility"] = df["wr_sum"] - df["wf_sum"]

    df["vocab_low"] = (df["wr_sum"] <= 7).astype(int)
    df["vocab_high"] = (df["wr_sum"] >= 11).astype(int)

    # 3-6. Interaction
    df["age_edu"] = df["age_ord"] * df["education"]
    df["young_low_edu"] = df["is_young"] * df["edu_low"]
    df["young_single"] = df["is_young"] * df["is_single"]
    df["old_married"] = df["is_old"] * df["is_married"]
    df["old_high_edu"] = df["is_old"] * df["edu_high"]
    df["teenager_low_edu"] = df["is_teenager"] * df["edu_low"]
    df["vocab_edu"] = df["wr_sum"] * df["education"]

    return df


# ============================================================
# 4. Target Encoding (CV leak 방지)
# ============================================================
print("\n" + "=" * 60)
print("🎯 4. Target Encoding 함수 정의")
print("=" * 60)


def target_encode(train_df, val_df, test_df, col, target_col, smoothing=100):
    global_mean = train_df[target_col].mean()
    agg = train_df.groupby(col)[target_col].agg(["mean", "count"])

    agg["te"] = (agg["count"] * agg["mean"] + smoothing * global_mean) / (agg["count"] + smoothing)
    te_map = agg["te"].to_dict()

    train_te = train_df[col].map(te_map).fillna(global_mean)
    val_te = val_df[col].map(te_map).fillna(global_mean)
    test_te = test_df[col].map(te_map).fillna(global_mean)

    return train_te.values, val_te.values, test_te.values


def create_all_target_encodings(train_df, val_df, test_df, target_col="voted_bin"):
    te_dict = {"train": {}, "val": {}, "test": {}}

    # ✅ (수정 #3) smoothing 크게
    SM_SINGLE = 100
    SM_2WAY = 200
    SM_3WAY = 500

    # 단일 컬럼 TE
    single_cols = ["age_group", "race", "religion"]
    for col in single_cols:
        tr, va, te = target_encode(train_df, val_df, test_df, col, target_col, smoothing=SM_SINGLE)
        te_dict["train"][f"{col}_te"] = tr
        te_dict["val"][f"{col}_te"] = va
        te_dict["test"][f"{col}_te"] = te

    # 복합 컬럼 TE (문자열 결합)
    for df in [train_df, val_df, test_df]:
        df["age_edu_cat"] = df["age_group"].astype(str) + "_" + df["education"].astype(str)
    tr, va, te = target_encode(train_df, val_df, test_df, "age_edu_cat", target_col, smoothing=SM_2WAY)
    te_dict["train"]["age_edu_te"] = tr
    te_dict["val"]["age_edu_te"] = va
    te_dict["test"]["age_edu_te"] = te

    for df in [train_df, val_df, test_df]:
        df["age_married_cat"] = df["age_group"].astype(str) + "_" + df["married"].astype(str)
    tr, va, te = target_encode(train_df, val_df, test_df, "age_married_cat", target_col, smoothing=SM_2WAY)
    te_dict["train"]["age_married_te"] = tr
    te_dict["val"]["age_married_te"] = va
    te_dict["test"]["age_married_te"] = te

    for df in [train_df, val_df, test_df]:
        df["age_race_cat"] = df["age_group"].astype(str) + "_" + df["race"].astype(str)
    tr, va, te = target_encode(train_df, val_df, test_df, "age_race_cat", target_col, smoothing=SM_2WAY)
    te_dict["train"]["age_race_te"] = tr
    te_dict["val"]["age_race_te"] = va
    te_dict["test"]["age_race_te"] = te

    for df in [train_df, val_df, test_df]:
        df["age_edu_married_cat"] = (
            df["age_group"].astype(str)
            + "_"
            + df["education"].astype(str)
            + "_"
            + df["married"].astype(str)
        )
    tr, va, te = target_encode(train_df, val_df, test_df, "age_edu_married_cat", target_col, smoothing=SM_3WAY)
    te_dict["train"]["age_edu_married_te"] = tr
    te_dict["val"]["age_edu_married_te"] = va
    te_dict["test"]["age_edu_married_te"] = te

    return te_dict


# ============================================================
# 5. 피처 컬럼 정의
# ============================================================
print("\n" + "=" * 60)
print("📋 5. 피처 컬럼 정의")
print("=" * 60)

qa_cols = [f"Q{c}A" for c in "abcdefghijklmnopqrst"]
qe_log_cols = [f"Q{c}E_log" for c in "abcdefghijklmnopqrst"]
wr_cols = [f"wr_{i:02d}" for i in range(1, 14)]
wf_cols = [f"wf_{i:02d}" for i in range(1, 4)]

num_features = (
    qa_cols
    + qe_log_cols
    + wr_cols
    + wf_cols
    + [
        "age_ord", "education", "married", "urban", "engnat", "familysize", "hand",
        "is_teenager", "is_young", "is_middle", "is_old",
        "edu_low", "edu_high", "is_single", "is_married",
        "is_urban", "is_rural", "is_english_native", "is_male",
        "qa_mean", "qa_std", "qa_min", "qa_max", "qa_range",
        "qa_extreme_ratio", "qa_neutral_ratio", "qa_negative_ratio", "qa_positive_ratio",
        "qa_all_same",
        "qe_log_mean", "qe_log_std", "qe_log_min", "qe_log_max",
        "qe_fast_ratio", "qe_slow_ratio", "qe_total_log",
        "is_careless",
        "tp_missing_ratio", "tp_mean", "tp_std",
        "extraversion", "agreeableness", "conscientiousness", "neuroticism", "openness",
        "wr_sum", "wf_sum", "word_credibility", "vocab_low", "vocab_high",
        "age_edu", "young_low_edu", "young_single", "old_married", "old_high_edu",
        "teenager_low_edu", "vocab_edu",
    ]
)

te_features = [
    "age_group_te", "race_te", "religion_te",
    "age_edu_te", "age_married_te", "age_race_te", "age_edu_married_te"
]

cat_features = ["gender", "race", "religion"]

print(f"수치형 피처: {len(num_features)}")
print(f"Target Encoding: {len(te_features)}")
print(f"범주형 피처: {len(cat_features)}")
print(f"총: {len(num_features) + len(te_features) + len(cat_features)}")


# ============================================================
# 6. Dataset & Model
# ============================================================
print("\n" + "=" * 60)
print("🏗️ 6. Dataset & Model 정의")
print("=" * 60)


class TabDataset(Dataset):
    def __init__(self, X_num, X_cat, y=None):
        self.X_num = torch.tensor(X_num, dtype=torch.float32)
        self.X_cat = torch.tensor(X_cat, dtype=torch.long)
        self.y = None if y is None else torch.tensor(y, dtype=torch.float32).unsqueeze(1)

    def __len__(self):
        return len(self.X_num)

    def __getitem__(self, idx):
        if self.y is None:
            return self.X_num[idx], self.X_cat[idx]
        return self.X_num[idx], self.X_cat[idx], self.y[idx]


class SimpleMLPWithEmbedding(nn.Module):
    def __init__(self, num_features, cat_dims, embed_dim=8, hidden_dims=[256, 128, 64], dropout=0.3):
        super().__init__()

        self.embeddings = nn.ModuleList([nn.Embedding(dim + 1, embed_dim) for dim in cat_dims])

        total_embed_dim = len(cat_dims) * embed_dim
        input_dim = num_features + total_embed_dim

        layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout),
            ])
            prev_dim = hidden_dim

        self.mlp = nn.Sequential(*layers)
        self.output = nn.Linear(hidden_dims[-1], 1)

        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Embedding):
                nn.init.normal_(m.weight, std=0.02)

    def forward(self, x_num, x_cat):
        cat_embeds = [emb(x_cat[:, i]) for i, emb in enumerate(self.embeddings)]
        cat_embeds = torch.cat(cat_embeds, dim=1)
        x = torch.cat([x_num, cat_embeds], dim=1)
        x = self.mlp(x)
        return self.output(x)


# ============================================================
# 7. 학습 함수
# ============================================================
def train_fold(model, train_loader, val_loader, y_train, val_y, device, epochs=EPOCHS, patience=PATIENCE):
    """
    ✅ (수정 #1) pos_weight를 y_train 기준으로 계산
    """
    model.to(device)

    pos_ratio = float(np.mean(y_train))
    pos_weight = torch.tensor([(1 - pos_ratio) / (pos_ratio + 1e-6)], device=device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", factor=0.5, patience=3)

    best_auc = 0.0
    best_state = None
    no_improve = 0

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0

        for X_num, X_cat, y in train_loader:
            X_num, X_cat, y = X_num.to(device), X_cat.to(device), y.to(device)

            optimizer.zero_grad()
            out = model(X_num, X_cat)
            loss = criterion(out, y)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            train_loss += loss.item()

        model.eval()
        val_preds = []
        with torch.no_grad():
            for X_num, X_cat, _ in val_loader:
                X_num, X_cat = X_num.to(device), X_cat.to(device)
                out = torch.sigmoid(model(X_num, X_cat))
                val_preds.append(out.cpu().numpy())

        val_preds = np.concatenate(val_preds).ravel()
        val_auc = roc_auc_score(val_y, val_preds)

        scheduler.step(val_auc)

        if val_auc > best_auc + 1e-5:
            best_auc = val_auc
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            no_improve = 0
        else:
            no_improve += 1

        if (epoch + 1) % 10 == 0:
            print(f"    Epoch {epoch+1}: loss={train_loss/len(train_loader):.4f}, val_auc={val_auc:.5f}, best={best_auc:.5f}")

        if no_improve >= patience:
            print(f"    Early stopping at epoch {epoch+1}")
            break

    if best_state is not None:
        model.load_state_dict(best_state)

    return model, best_auc


def predict(model, loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for batch in loader:
            if len(batch) == 2:
                X_num, X_cat = batch
            else:
                X_num, X_cat, _ = batch
            X_num, X_cat = X_num.to(device), X_cat.to(device)
            out = torch.sigmoid(model(X_num, X_cat))
            preds.append(out.cpu().numpy())
    return np.concatenate(preds).ravel()


# ============================================================
# 8. 메인 학습 루프
# ============================================================
def main():
    print("\n" + "=" * 60)
    print("🚀 8. 학습 시작")
    print("=" * 60)

    set_seed(SEED)

    train_clean = clean_data(train_raw, is_train=True)
    test_clean = clean_data(test_raw, is_train=False)

    oof_preds = np.zeros(len(train_clean))
    test_preds = np.zeros(len(test_clean))
    fold_aucs = []

    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

    for fold, (train_idx, val_idx) in enumerate(skf.split(train_clean, train_clean["voted_bin"])):
        print(f"\n{'='*50}")
        print(f"📂 Fold {fold + 1}/{N_FOLDS}")
        print(f"{'='*50}")

        train_fold_df = train_clean.iloc[train_idx].copy().reset_index(drop=True)
        val_fold_df = train_clean.iloc[val_idx].copy().reset_index(drop=True)
        test_fold_df = test_clean.copy()

        train_fe = build_features(train_fold_df, is_train=True)
        val_fe = build_features(val_fold_df, is_train=True)
        test_fe = build_features(test_fold_df, is_train=False)

        te_dict = create_all_target_encodings(train_fe, val_fe, test_fe, "voted_bin")

        # 수치형 준비
        all_num_features = num_features + te_features

        X_train = train_fe[num_features].copy()
        X_val = val_fe[num_features].copy()
        X_test = test_fe[num_features].copy()

        for te_name in te_features:
            X_train[te_name] = te_dict["train"][te_name]
            X_val[te_name] = te_dict["val"][te_name]
            X_test[te_name] = te_dict["test"][te_name]

        # NaN 처리 (train 중앙값)
        for col in all_num_features:
            median_val = X_train[col].median()
            if pd.isna(median_val):
                median_val = 0
            X_train[col] = X_train[col].fillna(median_val)
            X_val[col] = X_val[col].fillna(median_val)
            X_test[col] = X_test[col].fillna(median_val)

        # ✅ (수정 #4) QuantileTransformer로 스케일링 교체
        scaler = QuantileTransformer(
            n_quantiles=2000,
            output_distribution="normal",
            random_state=SEED
        )
        X_train_scaled = scaler.fit_transform(X_train.values)
        X_val_scaled = scaler.transform(X_val.values)
        X_test_scaled = scaler.transform(X_test.values)

        # 범주형 인코딩
        cat_dims = []
        X_cat_train_list = []
        X_cat_val_list = []
        X_cat_test_list = []

        for col in cat_features:
            le = LabelEncoder()

            train_col = train_fe[col].fillna("__NAN__").astype(str)
            val_col = val_fe[col].fillna("__NAN__").astype(str)
            test_col = test_fe[col].fillna("__NAN__").astype(str)

            all_values = list(set(train_col.unique()) | set(val_col.unique()) | set(test_col.unique()))
            if "__UNK__" not in all_values:
                all_values.append("__UNK__")

            le.fit(all_values)
            cat_dims.append(len(le.classes_))

            X_cat_train_list.append(le.transform(train_col))
            X_cat_val_list.append(le.transform(val_col.apply(lambda x: x if x in le.classes_ else "__UNK__")))
            X_cat_test_list.append(le.transform(test_col.apply(lambda x: x if x in le.classes_ else "__UNK__")))

        X_cat_train = np.stack(X_cat_train_list, axis=1)
        X_cat_val = np.stack(X_cat_val_list, axis=1)
        X_cat_test = np.stack(X_cat_test_list, axis=1)

        y_train = train_fe["voted_bin"].values.astype(np.float32)
        y_val = val_fe["voted_bin"].values.astype(np.float32)

        train_ds = TabDataset(X_train_scaled, X_cat_train, y_train)
        val_ds = TabDataset(X_val_scaled, X_cat_val, y_val)
        test_ds = TabDataset(X_test_scaled, X_cat_test)

        # ✅ (수정 #2) drop_last=False
        train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)
        val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
        test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

        model = SimpleMLPWithEmbedding(
            num_features=len(all_num_features),
            cat_dims=cat_dims,
            embed_dim=8,
            hidden_dims=[256, 128, 64],
            dropout=0.3
        )

        print(f"    모델 파라미터: {sum(p.numel() for p in model.parameters()):,}")

        model, fold_auc = train_fold(model, train_loader, val_loader, y_train, y_val, DEVICE)
        fold_aucs.append(fold_auc)

        oof_preds[val_idx] = predict(model, val_loader, DEVICE)
        test_preds += predict(model, test_loader, DEVICE) / N_FOLDS

        print(f"  ✅ Fold {fold + 1} AUC: {fold_auc:.5f}")

    final_oof_auc = roc_auc_score(train_clean["voted_bin"], oof_preds)

    print("\n" + "=" * 60)
    print("🎉 최종 결과")
    print("=" * 60)
    print(f"🏆 OOF AUC: {final_oof_auc:.5f}")
    print(f"📊 Fold AUCs: {[f'{x:.5f}' for x in fold_aucs]}")
    print(f"📊 Mean ± Std: {np.mean(fold_aucs):.5f} ± {np.std(fold_aucs):.5f}")

    sub = pd.DataFrame({
        "index": test_raw["index"] if "index" in test_raw.columns else range(len(test_raw)),
        "voted": test_preds
    })
    sub.to_csv("submission_16_new_6th_test.csv", index=False)
    print("\n💾 저장 완료: submission_test.csv")
    print(f"   예측 범위: [{test_preds.min():.4f}, {test_preds.max():.4f}]")
    print(f"   예측 평균: {test_preds.mean():.4f}")

    return final_oof_auc


if __name__ == "__main__":
    final_auc = main()




🎯 4. Target Encoding 함수 정의

📋 5. 피처 컬럼 정의
수치형 피처: 113
Target Encoding: 7
범주형 피처: 3
총: 123

🏗️ 6. Dataset & Model 정의

🚀 8. 학습 시작

📂 Fold 1/5
    모델 파라미터: 79,449
    Epoch 10: loss=0.5035, val_auc=0.76314, best=0.76376
    Early stopping at epoch 15
  ✅ Fold 1 AUC: 0.76376

📂 Fold 2/5
    모델 파라미터: 79,449
    Epoch 10: loss=0.5038, val_auc=0.77367, best=0.77367
    Early stopping at epoch 19
  ✅ Fold 2 AUC: 0.77425

📂 Fold 3/5
    모델 파라미터: 79,449
    Epoch 10: loss=0.4997, val_auc=0.76232, best=0.76232
    Early stopping at epoch 19
  ✅ Fold 3 AUC: 0.76260

📂 Fold 4/5
    모델 파라미터: 79,449
    Epoch 10: loss=0.5000, val_auc=0.76630, best=0.76637
    Early stopping at epoch 19
  ✅ Fold 4 AUC: 0.76741

📂 Fold 5/5
    모델 파라미터: 79,449
    Epoch 10: loss=0.5044, val_auc=0.77263, best=0.77348
    Early stopping at epoch 17
  ✅ Fold 5 AUC: 0.77348

🎉 최종 결과
🏆 OOF AUC: 0.76806
📊 Fold AUCs: ['0.76376', '0.77425', '0.76260', '0.76741', '0.77348']
📊 Mean ± Std: 0.76830 ± 0.00482

💾 저장 완료: submission_t

## 5. Dataset & 모델 정의


In [10]:
class TabDataset(Dataset):
    def __init__(self, X_num, X_cat, y=None):
        self.X_num = torch.tensor(X_num, dtype=torch.float32)
        self.X_cat = torch.tensor(X_cat, dtype=torch.long)
        self.y = None if y is None else torch.tensor(y, dtype=torch.float32).unsqueeze(1)

    def __len__(self):
        return len(self.X_num)

    def __getitem__(self, idx):
        if self.y is None:
            return self.X_num[idx], self.X_cat[idx]
        return self.X_num[idx], self.X_cat[idx], self.y[idx]


class SimpleMLPWithEmbedding(nn.Module):
    def __init__(self, num_features, cat_dims, embed_dim=8, hidden_dims=[256, 128, 64], dropout=0.3):
        super().__init__()

        self.embeddings = nn.ModuleList([nn.Embedding(dim + 1, embed_dim) for dim in cat_dims])

        total_embed_dim = len(cat_dims) * embed_dim
        input_dim = num_features + total_embed_dim

        layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout),
            ])
            prev_dim = hidden_dim

        self.mlp = nn.Sequential(*layers)
        self.output = nn.Linear(hidden_dims[-1], 1)

        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Embedding):
                nn.init.normal_(m.weight, std=0.02)

    def forward(self, x_num, x_cat):
        cat_embeds = [emb(x_cat[:, i]) for i, emb in enumerate(self.embeddings)]
        cat_embeds = torch.cat(cat_embeds, dim=1)
        x = torch.cat([x_num, cat_embeds], dim=1)
        x = self.mlp(x)
        return self.output(x)


# ============================================================
# 7. 학습 함수
# ============================================================



## 6. 학습/평가 함수


In [11]:
def train_fold(model, train_loader, val_loader, y_train, val_y, device, epochs=EPOCHS, patience=PATIENCE):
    """
    ✅ (수정 #1) pos_weight를 y_train 기준으로 계산
    """
    model.to(device)

    pos_ratio = float(np.mean(y_train))
    pos_weight = torch.tensor([(1 - pos_ratio) / (pos_ratio + 1e-6)], device=device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", factor=0.5, patience=3)

    best_auc = 0.0
    best_state = None
    no_improve = 0

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0

        for X_num, X_cat, y in train_loader:
            X_num, X_cat, y = X_num.to(device), X_cat.to(device), y.to(device)

            optimizer.zero_grad()
            out = model(X_num, X_cat)
            loss = criterion(out, y)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            train_loss += loss.item()

        model.eval()
        val_preds = []
        with torch.no_grad():
            for X_num, X_cat, _ in val_loader:
                X_num, X_cat = X_num.to(device), X_cat.to(device)
                out = torch.sigmoid(model(X_num, X_cat))
                val_preds.append(out.cpu().numpy())

        val_preds = np.concatenate(val_preds).ravel()
        val_auc = roc_auc_score(val_y, val_preds)

        scheduler.step(val_auc)

        if val_auc > best_auc + 1e-5:
            best_auc = val_auc
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            no_improve = 0
        else:
            no_improve += 1

        if (epoch + 1) % 10 == 0:
            print(f"    Epoch {epoch+1}: loss={train_loss/len(train_loader):.4f}, val_auc={val_auc:.5f}, best={best_auc:.5f}")

        if no_improve >= patience:
            print(f"    Early stopping at epoch {epoch+1}")
            break

    if best_state is not None:
        model.load_state_dict(best_state)

    return model, best_auc


def predict(model, loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for batch in loader:
            if len(batch) == 2:
                X_num, X_cat = batch
            else:
                X_num, X_cat, _ = batch
            X_num, X_cat = X_num.to(device), X_cat.to(device)
            out = torch.sigmoid(model(X_num, X_cat))
            preds.append(out.cpu().numpy())
    return np.concatenate(preds).ravel()


# ============================================================
# 8. 메인 학습 루프
# ============================================================
def main():
    print("\n" + "=" * 60)
    print("🚀 8. 학습 시작")
    print("=" * 60)

    set_seed(SEED)

    train_clean = clean_data(train_raw, is_train=True)
    test_clean = clean_data(test_raw, is_train=False)

    oof_preds = np.zeros(len(train_clean))
    test_preds = np.zeros(len(test_clean))
    fold_aucs = []

    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

    for fold, (train_idx, val_idx) in enumerate(skf.split(train_clean, train_clean["voted_bin"])):
        print(f"\n{'='*50}")
        print(f"📂 Fold {fold + 1}/{N_FOLDS}")
        print(f"{'='*50}")

        train_fold_df = train_clean.iloc[train_idx].copy().reset_index(drop=True)
        val_fold_df = train_clean.iloc[val_idx].copy().reset_index(drop=True)
        test_fold_df = test_clean.copy()

        train_fe = build_features(train_fold_df, is_train=True)
        val_fe = build_features(val_fold_df, is_train=True)
        test_fe = build_features(test_fold_df, is_train=False)

        te_dict = create_all_target_encodings(train_fe, val_fe, test_fe, "voted_bin")

        # 수치형 준비
        all_num_features = num_features + te_features

        X_train = train_fe[num_features].copy()
        X_val = val_fe[num_features].copy()
        X_test = test_fe[num_features].copy()

        for te_name in te_features:
            X_train[te_name] = te_dict["train"][te_name]
            X_val[te_name] = te_dict["val"][te_name]
            X_test[te_name] = te_dict["test"][te_name]

        # NaN 처리 (train 중앙값)
        for col in all_num_features:
            median_val = X_train[col].median()
            if pd.isna(median_val):
                median_val = 0
            X_train[col] = X_train[col].fillna(median_val)
            X_val[col] = X_val[col].fillna(median_val)
            X_test[col] = X_test[col].fillna(median_val)

        # ✅ (수정 #4) QuantileTransformer로 스케일링 교체
        scaler = QuantileTransformer(
            n_quantiles=2000,
            output_distribution="normal",
            random_state=SEED
        )
        X_train_scaled = scaler.fit_transform(X_train.values)
        X_val_scaled = scaler.transform(X_val.values)
        X_test_scaled = scaler.transform(X_test.values)

        # 범주형 인코딩
        cat_dims = []
        X_cat_train_list = []
        X_cat_val_list = []
        X_cat_test_list = []

        for col in cat_features:
            le = LabelEncoder()

            train_col = train_fe[col].fillna("__NAN__").astype(str)
            val_col = val_fe[col].fillna("__NAN__").astype(str)
            test_col = test_fe[col].fillna("__NAN__").astype(str)

            all_values = list(set(train_col.unique()) | set(val_col.unique()) | set(test_col.unique()))
            if "__UNK__" not in all_values:
                all_values.append("__UNK__")

            le.fit(all_values)
            cat_dims.append(len(le.classes_))

            X_cat_train_list.append(le.transform(train_col))
            X_cat_val_list.append(le.transform(val_col.apply(lambda x: x if x in le.classes_ else "__UNK__")))
            X_cat_test_list.append(le.transform(test_col.apply(lambda x: x if x in le.classes_ else "__UNK__")))

        X_cat_train = np.stack(X_cat_train_list, axis=1)
        X_cat_val = np.stack(X_cat_val_list, axis=1)
        X_cat_test = np.stack(X_cat_test_list, axis=1)

        y_train = train_fe["voted_bin"].values.astype(np.float32)
        y_val = val_fe["voted_bin"].values.astype(np.float32)

        train_ds = TabDataset(X_train_scaled, X_cat_train, y_train)
        val_ds = TabDataset(X_val_scaled, X_cat_val, y_val)
        test_ds = TabDataset(X_test_scaled, X_cat_test)

        # ✅ (수정 #2) drop_last=False
        train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)
        val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
        test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

        model = SimpleMLPWithEmbedding(
            num_features=len(all_num_features),
            cat_dims=cat_dims,
            embed_dim=8,
            hidden_dims=[256, 128, 64],
            dropout=0.3
        )

        print(f"    모델 파라미터: {sum(p.numel() for p in model.parameters()):,}")

        model, fold_auc = train_fold(model, train_loader, val_loader, y_train, y_val, DEVICE)
        fold_aucs.append(fold_auc)

        oof_preds[val_idx] = predict(model, val_loader, DEVICE)
        test_preds += predict(model, test_loader, DEVICE) / N_FOLDS

        print(f"  ✅ Fold {fold + 1} AUC: {fold_auc:.5f}")

    final_oof_auc = roc_auc_score(train_clean["voted_bin"], oof_preds)

    print("\n" + "=" * 60)
    print("🎉 최종 결과")
    print("=" * 60)
    print(f"🏆 OOF AUC: {final_oof_auc:.5f}")
    print(f"📊 Fold AUCs: {[f'{x:.5f}' for x in fold_aucs]}")
    print(f"📊 Mean ± Std: {np.mean(fold_aucs):.5f} ± {np.std(fold_aucs):.5f}")

    sub = pd.DataFrame({
        "index": test_raw["index"] if "index" in test_raw.columns else range(len(test_raw)),
        "voted": test_preds
    })
    sub.to_csv("submission_16_new_6th_test.csv", index=False)
    print("\n💾 저장 완료: submission_test.csv")
    print(f"   예측 범위: [{test_preds.min():.4f}, {test_preds.max():.4f}]")
    print(f"   예측 평균: {test_preds.mean():.4f}")

    return final_oof_auc



## 7. 실행 및 제출


In [12]:
if __name__ == "__main__":
    final_auc = main()




🚀 8. 학습 시작

📂 Fold 1/5
    모델 파라미터: 79,449
    Epoch 10: loss=0.5035, val_auc=0.76314, best=0.76376
    Early stopping at epoch 15
  ✅ Fold 1 AUC: 0.76376

📂 Fold 2/5
    모델 파라미터: 79,449
    Epoch 10: loss=0.5038, val_auc=0.77367, best=0.77367
    Early stopping at epoch 19
  ✅ Fold 2 AUC: 0.77425

📂 Fold 3/5
    모델 파라미터: 79,449
    Epoch 10: loss=0.4997, val_auc=0.76232, best=0.76232
    Early stopping at epoch 19
  ✅ Fold 3 AUC: 0.76260

📂 Fold 4/5
    모델 파라미터: 79,449
    Epoch 10: loss=0.5000, val_auc=0.76630, best=0.76637
    Early stopping at epoch 19
  ✅ Fold 4 AUC: 0.76741

📂 Fold 5/5
    모델 파라미터: 79,449
    Epoch 10: loss=0.5044, val_auc=0.77263, best=0.77348
    Early stopping at epoch 17
  ✅ Fold 5 AUC: 0.77348

🎉 최종 결과
🏆 OOF AUC: 0.76806
📊 Fold AUCs: ['0.76376', '0.77425', '0.76260', '0.76741', '0.77348']
📊 Mean ± Std: 0.76830 ± 0.00482

💾 저장 완료: submission_test.csv
   예측 범위: [0.1048, 0.9989]
   예측 평균: 0.5160
